# 日本国債損益モニタリング

## 概要

日本国債の時価を計算し，各基準日の損益をExcelで集計するツールである．

財務省が公表している日本国債（以下，単に国債）の入札結果をもとに，2024年4月から2025年3月までの月次で仮想の国債ポジション明細を作成する．明細は国債の新規購入・中途売却・満期償還も含む．

次に**財務省が日次で公表している国債の最終利回りから，QuantLib-PythonのFixedRateBondHelperクラスを用いて割引カーブを引く**．この時，元データの最終利回りについて以下を仮定する：

- 各年限グリッドごとに選定する代表銘柄はパー価格である，
    - ３年の最終利回りを計算する際に用いる残存3年の5年債の価格が100円である，
- 半年複利ベースの最終利回りを上記パー債の半年単利クーポンレートで近似できる．

ポシションを割引いて**時価と修正デュレーションを計算し**，明細に追加してcsvファイルとして出力する．

時価情報が追加された明細をPower BIに取り込み、評価損益をテーブル形式で集計する．テーブルには以下の項目を載せ，断面の時点をタイル形式で選択できるようにする。

**2024年12月 断面**

| 分類 | 銘柄名 | 簿価残高 | クーポン | 残存年限 | 修正Dur | 評価損益 | 評価損益_前Q比 | 実現損益 | 総合損益 | 評価損益_前Q |
| --- | --- | ---: | ---: | ---: | ---: | ---: | ---: | ---: | ---: | ---: |
| **短期** | 第448回利付国庫債券（2年） | 300 | 0.04% | 0.34| 0.32 | ▲0 | ▲0 | | ▲0 | ▲0 |
| **中期** | 第142回利付国庫債券（5年） | | | | | | | ▲10 | ▲10 | ▲10|
| | 第153回利付国庫債券（5年） | 10,000 | 0.04% | 2.47 | 2.45 | ▲138 | ▲45 | | ▲45 | ▲93 |
| | 第169回利付国庫債券（5年） | 1,400 | 0.51% | 4.22 | 4.15 | ▲9 | ▲11 | | ▲11 | 3 |
| | 第170回利付国庫債券（5年） | 500 | 0.61% | 4.47 | 4.39 | ▲2 | ▲6 | | ▲6 | 3 |
| **長期** | 第347回利付国庫債券（10年） | 10,000 | 0.07% | 2.47 | 2.45 | ▲65 | ▲23 | | ▲23 | ▲41 |
| | 第375回利付国庫債券（10年） | 10,000 | 0.29% | 9.47 | 9.03 | ▲13 | ▲18 | | ▲18 | 6 |
| **超長期** | 第95回利付国庫債券（20年） | 5,000 | 2.29% | 2.47 | 2.40 | 422 | ▲158 | | ▲158 | 580 |
| | 第145回利付国庫債券（20年） | 1,000 | 1.69% | 8.47 | 7.89 | 601 | ▲251 | | ▲251 | 852 |
| 合計 | | 38,200 | 1.11% | 4.30 | 4.10 | 797 | ▲512 | ▲10 | ▲522 | 1,300 |


## 実装

### ライブラリインポート

In [ ]:
import re #和暦→西暦変換用

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import QuantLib as ql

### 仮想の国債ポジション明細データの読み込み

In [ ]:
historical_meisai = pd.read_csv('Input//raw//日本国債明細.csv')
historical_meisai['基準日'] = pd.to_datetime(historical_meisai['基準日'], format='%Y/%m/%d')
historical_meisai['発行日'] = pd.to_datetime(historical_meisai['発行日'], format='%Y/%m/%d')
historical_meisai['償還日'] = pd.to_datetime(historical_meisai['償還日'], format='%Y/%m/%d')
historical_meisai.set_index(['基準日', '銘柄'], inplace=True)

# 基準日のリストの抽出
eval_dates = historical_meisai.index.get_level_values('基準日').unique()

display(historical_meisai)

### 日本国債コンベンションの定義

In [ ]:
tenor = ql.Period(ql.Semiannual)     # 利払間隔
calendar = ql.Japan()                # 休祝日カレンダー
convention = ql.ModifiedFollowing    # 営業日調整
day_count = ql.Thirty360(convention) # 日数計算
rule = ql.DateGeneration.Backward    # 日付生成のルール
end_of_month = False                 # 月末日ロール
settlemant_days = 1                  # 決済日数

### 和暦を西暦に変換する関数の定義

In [ ]:
def convert_japanese_era_to_gregorian(date_str: str) -> str | None:
    """和暦の日付を西暦に変換する関数"""
    
    # 元号と対応する西暦の開始年
    era_map: dict[str, int] = {
        "S": 1925,  # 昭和1年は1926年
        "H": 1988,  # 平成1年は1989年
        "R": 2018   # 令和1年は2019年
    }

    # 正規表現で元号・年・月・日を抽出
    match: re.Match | None = re.match(r"(S|H|R)(\d+)\.(\d+)\.(\d+)", date_str)
    
    if match:
        era: str
        year: int
        month: int
        day: int
        
        era, year_str, month_str, day_str = match.groups()
        year = int(year_str)
        month = int(month_str)
        day = int(day_str)

        # 西暦の計算
        gregorian_year: int = era_map[era] + year
        
        return f"{gregorian_year}.{month}.{day}"
    
    return None  # 変換できない場合

### 年限別の最終利回りデータの読み込み

In [ ]:
historical_ytms = pd.read_csv('Input//jgbcm_all.csv', encoding='shift-jis', header=1)
historical_ytms['基準日'] = historical_ytms['基準日'].apply(convert_japanese_era_to_gregorian)
historical_ytms['基準日'] = pd.to_datetime(historical_ytms['基準日'], format='%Y.%m.%d')
historical_ytms.set_index('基準日', inplace=True)

display(historical_ytms)

### 各基準日で最終利回りから割引カーブを引いて時価を計算

- 基準日についてfor文を展開
    - csvファイルから基準日の最終利回りを取得
    - 基準日の各年限についてfor文を展開
        - 該当年限の代表銘柄について**パー価格を仮定する**ことでFixedRateBondHelperクラスのhelperオブジェクトを構築
    - helperのリストからLogLinearで補完した割引カーブを構築
    - csvファイルから明細を取得，Bondクラスを作成し時価と修正デュレーションを計算

In [ ]:
list_meisai = []

for eval_date in eval_dates:

    # 基準日の最終利回りを取得
    rates = historical_ytms.loc[eval_date]
    dict_rates = rates.to_dict()
    dict_rates = {int(k[:-1]): np.float64(v) for k, v in rates.items()} # 年限を整数に変換し、値をfloat64に変換

    # QuantLibに基準日を設定
    eval_date_ql = ql.Date.from_date(eval_date)
    ql.Settings.instance().evaluationDate = eval_date_ql

    helpers = []
    for tenor, rate in dict_rates.items():
        
        # 代表銘柄のCFスケジュールを作成
        schedule = ql.Schedule(
            eval_date_ql,
            calendar.adjust(eval_date_ql + ql.Period(tenor, ql.Years), convention),
            ql.Period(ql.Semiannual),  # 支払頻度（半年ごと）
            calendar,  # カレンダー（例：ql.Japan()）
            convention,  # 調整規則（例：ql.Following）
            convention,  # 終了日の調整規則
            rule,  # 日付生成ルール
            end_of_month  # 月末調整
            )

        price = 100. # 代表銘柄についてパー価格を仮定
        price_quote = ql.SimpleQuote(price)
        price_handle = ql.QuoteHandle(price_quote)
        
        # 債券ヘルパーの作成
        helper = ql.FixedRateBondHelper(
            price_handle,    # 債券価格
            settlemant_days, # 決済日数
            100.0,           # 額面
            schedule,        # 債券のスケジュール
            [rate / 100],    # クーポン率
            day_count,       # 日数計算方法
        )
        helpers.append(helper)
    
    # 割引カーブの構築
    discount_curve = ql.PiecewiseLogLinearDiscount(eval_date_ql, helpers, day_count)
    discount_handle = ql.YieldTermStructureHandle(discount_curve)
    engine = ql.DiscountingBondEngine(discount_handle)

    meisai = historical_meisai.loc[eval_date].copy()
    # 各銘柄の時価と修正デュレーションを計算
    for _ in range(len(meisai)):
        meigara = meisai.iloc[_]
        
        # 保有債券のCFスケジュールを作成
        schedule = ql.Schedule(
            ql.Date.from_date(meigara['発行日']),
            ql.Date.from_date(meigara['償還日']),
            ql.Period(ql.Semiannual),
            calendar,
            convention,
            convention,
            rule,
            end_of_month
            )

        # 保有債券の固定利付債券オブジェクトを作成
        bond = ql.FixedRateBond(
                settlemant_days,
                np.float64(meigara['簿価']),         
                schedule,
                [meigara['利率']],
                day_count
            )
        bond.setPricingEngine(engine)            
        meisai.loc[meisai.index[_], '時価'] = bond.NPV()
        
        # 簿価が0の場合は修正デュレーションを0に設定
        if bond.notional() == 0:
            meisai.loc[meisai.index[_], '修正デュレーション'] = 0.0
        else:
           # 経過利息を除いた価格の計算
            clean_price = ql.BondPrice(bond.cleanPrice(), ql.BondPrice.Clean)

            # 最終利回りの計算
            ytm = bond.bondYield(
                clean_price,
                day_count,
                ql.Compounded, 
                ql.Semiannual
            )
            # 金利オブジェクトの作成
            interest_rate = ql.InterestRate(
                ytm,
                day_count,
                ql.Compounded,
                ql.Semiannual
            )
            # 修正デュレーションの計算
            duration = ql.BondFunctions.duration(
                bond,
                interest_rate,
                ql.Duration.Modified
            )
            meisai.loc[meisai.index[_], '修正デュレーション'] = duration

    meisai['基準日'] = eval_date
    meisai.set_index('基準日', append=True, inplace=True)
    meisai = meisai.swaplevel(0, 1)
    list_meisai.append(meisai)

参考までに2025年3月31日時点の日本国債の割引カーブをプロットした．

In [ ]:
dates = discount_curve.dates()
discounts = [discount_curve.discount(date) for date in dates]
date_labels = [date.to_date() for date in dates]

df = pd.DataFrame({
    "Date": date_labels,
    "Discount": discounts
})
display(df)

plt.figure(figsize=(4, 3))
plt.plot(df["Date"], df["Discount"], marker='o', linestyle='-')
plt.title("Discount Curve (Piecewise Log Linear)")
plt.xlabel("Date")
plt.ylabel("Discount Factor")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## ファイルの出力

In [ ]:
pd.concat(list_meisai).to_csv('Output//国内債券明細_時価評価.csv', encoding='utf-8')